In [1]:
import tensorflow as  tf
import tensorflow.keras as keras
import numpy as np

In [2]:
keras.__version__

'2.4.0'

In [3]:
import os

# folder_path = "D:\yun\yun-python\딥러닝(kmooc 연습)\포켓몬 이미지"
folder_path = "D:\yun\yun-python\deeplearning\pokemon-train-image"
folder_list = os.listdir(folder_path)
print(folder_list)
print(f'Total number of categories: {len(folder_list)}')

counts = {}
for c in folder_list:
    counts[c] = len(os.listdir(os.path.join(folder_path, c)))
    
print(f'Total number of images in dataset: {sum(list(counts.values()))}')

# categories=['강철', '격투', '고스트', '노멀', '독', '드래곤', '땅', '물', '바위', '벌레', '불꽃', '악', '얼음', '에스퍼', '전기', '페어리', '풀']

categories=['bug', 'dragon', 'electricity', 'esp', 'evil', 'fairy', 'fight', 'fire', 'ghost', 'glue', 'ice', 'land', 'normal', 'posion', 'rock', 'steel', 'water']

['bug', 'dragon', 'electricity', 'esp', 'evil', 'fairy', 'fight', 'fire', 'ghost', 'grass', 'ice', 'land', 'normal', 'posion', 'rock', 'steel', 'water']
Total number of categories: 17
Total number of images in dataset: 55588


In [4]:
imbalanced = sorted(counts.items(), key = lambda x: x[1], reverse = True)
print(imbalanced)

[('water', 6410), ('normal', 5096), ('fire', 4101), ('bug', 4001), ('grass', 3916), ('land', 3448), ('esp', 3177), ('posion', 3086), ('electricity', 3046), ('rock', 2905), ('fight', 2904), ('ice', 2670), ('dragon', 2397), ('ghost', 2356), ('steel', 2305), ('evil', 2072), ('fairy', 1698)]


In [5]:
# caltech_dir = "..\deeplearning\pokemon-image"
caltech_dir = "pokemon-train-image"
categories=['bug', 'dragon', 'electricity', 'esp', 'evil', 'fairy', 'fight', 'fire', 'ghost', 'glue', 'ice', 'land', 'normal', 'posion', 'rock', 'steel', 'water']
nb_classes = len(categories)

In [6]:
# !dir .

In [9]:
# 이미지 전처리

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      validation_split=0.2,
      fill_mode='nearest')

generator = datagen.flow_from_directory(
        # 타깃 디렉터리
        caltech_dir,
        # 모든 이미지를 150 × 150 크기로 바꿉니다
        target_size=(150, 150),
        shuffle=True,
        batch_size=40,
        class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range = 40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=False,
                                 vertical_flip=True,
                                 fill_mode='nearest')

val_generator = val_datagen.flow_from_directory(
                                          caltech_dir,
                                          target_size=(150,150),
                                          batch_size=40,
                                          class_mode='categorical')

Found 55588 images belonging to 17 classes.
Found 55588 images belonging to 17 classes.


In [10]:
from keras.models import Sequential
from keras.layers import *
from keras import optimizers

In [11]:
# 모델 구성

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),padding='same', input_shape=(150,150,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(17, activation='softmax'))

In [10]:
# model.summary()

In [12]:
#모델 학습 설정
sgd = optimizers.SGD(learning_rate=0.01, momentum=0.9 , nesterov=True)
model.compile(loss='categorical_crossentropy',
             optimizer=sgd,
             metrics=['accuracy'])

In [13]:
# fig_generator
model.fit_generator(
    generator,
    steps_per_epoch=200,
    epochs=150,
    validation_data=val_generator,
    validation_steps=70)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/150
200/200 [==============================] - 694s 3s/step - loss: 2.7573 - accuracy: 0.1700 - val_loss: 2.6863 - val_accuracy: 0.1593
Epoch 2/150
200/200 [==============================] - 693s 3s/step - loss: 2.5684 - accuracy: 0.2130 - val_loss: 2.6398 - val_accuracy: 0.1868
Epoch 3/150
200/200 [==============================] - 725s 4s/step - loss: 2.5229 - accuracy: 0.2300 - val_loss: 2.5966 - val_accuracy: 0.2025
Epoch 4/150
200/200 [==============================] - 709s 4s/step - loss: 2.4876 - accuracy: 0.2311 - val_loss: 2.6535 - val_accuracy: 0.1882
Epoch 5/150
200/200 [==============================] - 679s 3s/step - loss: 2.4597 - accuracy: 0.2376 - val_loss: 2.5002 - val_accuracy: 0.2364
Epoch 6/150
200/200 [==============================] - 668s 3s/step - loss: 2.4257 - accuracy: 0.2519 - val_loss: 2.5457 - val_accuracy: 0.2029
Epoch 7/150
200/200 [==============================] - 671s 

KeyboardInterrupt: 